In [1]:
import pyaudio
import wave
import numpy as np
import librosa
import threading

In [2]:
#%matplotlib inline
import matplotlib
matplotlib.use( 'TkAgg' )
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [3]:
class StreamThread(threading.Thread):
    """Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition."""

    def __init__(self, 
                 chunk=300,
                 st_format=pyaudio.paInt16,
                 channels=1,
                 rate=44100,
                ):
        super(StreamThread, self).__init__()
        self._stop_event = threading.Event()
        self.data = np.zeros((1,))
        self.lock = threading.Lock()
        self.chunk = chunk
        self.st_format = st_format
        self.channels = channels
        self.rate = rate
        self.pyAudio = pyaudio.PyAudio()
        self.stream = self.pyAudio.open(format=self.st_format,
                                   channels=self.channels,
                                   rate=self.rate,
                                   input=True,
                                   frames_per_buffer=self.chunk,
                                  )
        
    def run(self):
        while not self.stopped():
            #with self.lock:
            self.data = self.get_stream_data()
    
    def stop(self):
        self.stream.stop_stream()
        self.stream.close()
        p.terminate()
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    def get_stream_data(self):
        data = self.stream.read(self.chunk)
        y=np.frombuffer(data, dtype=np.int16)
        z=np.asarray(y,dtype=float)
        S = librosa.feature.melspectrogram(z, sr=self.rate, n_mels=150)
        return librosa.power_to_db(S, ref=100)
        #return np.fft.rfft(y)

In [4]:
streamThread = StreamThread()
streamThread.start()

In [5]:
#%%timeit
#data=np.log(np.abs(streamThread.get_stream_data()[:,0]))
#np.asarray(np.round(data/np.amax(data)*255), dtype=int)
#len(data)
#streamThread.get_stream_data()[:,0]
#ydata=(streamThread.get_stream_data()[:,0])/1
#ydata#np.asarray(np.round(ydata), dtype=int)

In [6]:
def run():
    #fig, ax = plt.subplots(1, 1)
    #ax.set_aspect('equal')
    #ax.hold(True)
    #plt.show(False)
    #plt.draw()
    
    #background = fig.canvas.copy_from_bbox(ax.bbox)
    #x=np.linspace(0,len(streamThread.data), len(streamThread.data))
    #print(x)
    #points = ax.plot(x, streamThread.data)[0]
    #print(streamThread.data)
    fig = plt.figure()
    fig.show()
    fig.canvas.draw()
    while True:
        try:
            #with streamThread.lock:
                #x=np.linspace(0,len(streamThread.data),len(streamThread.data))
                #print(x)
                #points.set_data(x, streamThread.data)
                #fig.canvas.restore_region(background)
                #ax.draw_artist(points)
                #fig.canvas.blit(ax.bbox)
                #fig.canvas.draw()
            ax=plt.gca()
            ax.plot(streamThread.data) # plot something
            ax.set_xscale("log")
            fig.canvas.draw()
            plt.pause(0.0001)
            plt.clf()
                

        except KeyboardInterrupt:
            streamThread.stop()
            streamThread.join()
            break
    plt.close(fig)

In [7]:
def run_thre():
    fig, ax = plt.subplots()
    #freq = np.fft.rfftfreq(len(streamThread.data[:,0]), 1/streamThread.rate)
    xdata = np.linspace(0,len(streamThread.data[:,0]), len(streamThread.data[:,0]))
    ydata = streamThread.data[:,0]
    ln, = plt.plot(xdata, ydata)

    def init():
        #ax.set_yscale("log")
        ax.set_xscale("log")
        #ax.set_xlim(0, 2*np.pi)
        ax.set_ylim(0, 1)
        return ln,

    def update(frame):
        ydata=streamThread.data
        ydata[ydata<0]=0
        ydata=np.asarray([(1.+0.7*x/150)*y for x,y in zip(range(0,150),ydata)])
        c=np.amin(ydata)
        ydata=np.asarray([y-c for y in ydata])
        ydata=ydata/np.amax(ydata)
        #ydata=np.abs(ydata)
        #ydata=np.asarray(np.round(ydata/np.amax(ydata)*255), dtype=int)
        xdata = np.linspace(0,len(streamThread.data[:,0]), len(streamThread.data[:,0]))
        #ax.set_xlim(0,len(streamThread.data))
        #ax.set_ylim(0,max(streamThread.data)*1.1)
        ln.set_data(xdata, ydata)
        return ln,

    ani = FuncAnimation(fig, update,
                        init_func=init, blit=True)
    plt.show()

In [ ]:
run_thre()

/home/tassilo/Projects/RGBMuse/.RGBMuse_py/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
